In [1]:
#Importamos las librerias
import os
import csv
import pandas as pd
import polars as pl
import numpy as np
import datetime

In [2]:
# Datasets
file_list = ['Datasets/amazon_prime_titles-score.csv', 'Datasets/disney_plus_titles-score.csv', 'Datasets/hulu_titles-score (2).csv', 'Datasets/netflix_titles-score.csv']

# Se realizan las transformaciones solicitadas
data = pd.DataFrame()

# Se iteran los archivos csv del directorio Datasets
for file in file_list:
    file_name = os.path.basename(file)

    #Se toma el prefijo de cada plataforma
    file_prefix = file_name[0]

    # Se lee el archivo
    df = pd.read_csv(file)

    # Se realiza la primera trasnformacion
    df['id'] = file_prefix + df['show_id']

    # Se rellenan los nulos del rating
    df['rating'].fillna('G', inplace=True)

    # Se transforman las fechas
    df['date_added'] = pd.to_datetime(df['date_added']).dt.strftime('%Y-%m-%d')

    # Se convierte a minuscula todos los campos texto
    df = df.apply(lambda x: x.astype(str).str.lower())

    # Se divide la columna 'duration' para obtener una columna cuantitativa y otra categorica
    cat = df['duration'].str.split(" ", n = 1, expand =True) 
    df['duration_int']= cat[0] 
    df['duration_type']= cat[1] 
    # Se reemplazan los valores nulos por 0
    df['duration_int'] = pd.to_numeric(df['duration_int'], errors='coerce')
    df['duration_int'].fillna(value=0,inplace=True)
    # Se convierte a entero
    df = df.astype({"duration_int": int})

    # Transformaciones que se realizan para evitar errores
    df['score'] = pd.to_numeric(df["score"], errors='coerce')
    df["date_added"] = pd.to_datetime(df["date_added"], errors='coerce')
    df["release_year"] = pd.to_numeric(df["release_year"], errors='coerce')

      
    # Se van agregando los datasets a un dataset consolidado gral, para ser utilizado en las consultas de la fastapi
    data = pd.concat([data, df], ignore_index=True)

In [3]:
data.dtypes

show_id                  object
type                     object
title                    object
director                 object
cast                     object
country                  object
date_added       datetime64[ns]
release_year              int64
rating                   object
duration                 object
listed_in                object
description              object
score                     int64
id                       object
duration_int              int64
duration_type            object
dtype: object

In [4]:
data.tail(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,id,duration_int,duration_type
22996,s8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,pg,88 min,"children & family movies, comedies","dragged from civilian life, a former superhero...",7,ns8806,88,min
22997,s8807,movie,zubaan,mozez singh,"vicky kaushal, sarah-jane dias, raaghav chanan...",india,2019-03-02,2015,tv-14,111 min,"dramas, international movies, music & musicals",a scrappy but poor boy worms his way into a ty...,44,ns8807,111,min


In [5]:
# Se guarda el dataframe en un csv que va utilizar fastapi para realizar las consultas y como resguardo
data.to_csv('data_concatenada.csv', encoding='utf-8')

Declaración de las funciones requeridas y sus respectivos testeos.

1 - Cantidad de veces que aparece una keyword en el título de peliculas/series, por plataforma

In [6]:
def search_by_keyword(plataforma: str, keyword: str):
    
    # Se filtra el dataframe para obtener los registros que comienzan con el primer caracter del parametro plataforma
    result = data[data['id'].str.startswith(plataforma[0])]
    
    # Se filtra el dataframe según la keyword
    filtered_df = result[result['title'].str.contains(keyword)]

    #obtener la cantidad de veces que aparece la keyword en el título
    count = len(filtered_df)

    #imprimir el resultado
    print(f'La keyword {keyword} aparece {count} veces en el título de películas/series.')
    return {plataforma, count}

In [7]:
print(search_by_keyword('netflix','love'))

La keyword love aparece 196 veces en el título de películas/series.
{'netflix', 196}


2 - Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año

In [7]:
def movie_for_platform(plataforma: str, puntaje: int, year: int):
    
    # Se filtra el dataframe para obtener los registros que comienzan con el primer caracter del parametro plataforma
    filtered_data = data[(data['id'].str[0] == plataforma[0])] 
   
    # Se filtra el dataframe según la keyword
    filtered_data = filtered_data[(filtered_data['score'] > puntaje)] 
   
    # Se filtra el dataframe según el año
    filtered_data = filtered_data[(filtered_data['release_year'] == year)]
    
    return {"cantidad de peliculas": len(filtered_data)}

In [8]:
print(movie_for_platform('netflix',85,2010))

{'cantidad de peliculas': 25}


3 - La segunda película con mayor score para una plataforma determinada, según el orden alfabético de los títulos.

In [14]:
def get_second_highest_movie_by_platform(plataforma: str):
   
   # Se filtra el dataframe para obtener los registros que comienzan con el primer caracter del parametro plataforma
    filtered_df = data[data['id'].str.startswith(plataforma[0])]
    
    #ordenar el dataframe según la columna 'title' de forma ascendente
    sorted_df = filtered_df.sort_values(by='score', ascending=False)
    
    #obtener la segunda película con el mayor score
    second_highest_score = sorted_df.iloc[1]['score']
    titulo = sorted_df.iloc[1]["title"] 

    return {"Plataforma ":plataforma,"Titulo  ":titulo,"Score  ":second_highest_score}


In [15]:
print(get_second_highest_movie_by_platform('amazon'))

{'Plataforma ': 'amazon', 'Titulo  ': 'the lazarus effect', 'Score  ': 100}


4 - Película que más duró según año, plataforma y tipo de duración

In [17]:

def longest_movie( platform, duration_type, year,):
    
    # Se filtra el DataFrame para obtener solo las filas con el año, plataforma y tipo de duración específicos
    filtered_data = data.query("release_year == @year and id.str[0] == @platform[0] and duration_type == @duration_type")
   
    # Se ordena el DataFrame filtrado por la duración
    filtered_data = filtered_data.sort_values(by='duration_int', ascending=False)
   
    #Se valida si hay alguna pelicula con los criterios especificados
    if filtered_data.empty:
      return None
    else:
      # Se devuelve el título y la fila completa de la película
      title = filtered_data.iloc[0]['title']
      duration = filtered_data.iloc[0]['duration_int']
      return {"Titulo  ":title, "Duration  ": duration}


In [18]:
print(longest_movie('netflix','min',2016))

{'Titulo  ': 'sairat', 'Duration  ': 173}


5 - Cantidad de series y películas por rating

In [128]:
def count_shows_movies_by_classification(classification):
    # Se filtra el DataFrame para obtener solo las filas con la clasificación específica
    filtered_data = data.query("rating == @classification")
    # Se cuenta la cantidad de filas en el DataFrame filtrado
    count = filtered_data.size
    return count


In [129]:
print(count_shows_movies_by_classification('18+'))

19888
